# Load the Data

In [ ]:
import pandas as pd

data = pd.read_csv("Date_overlap/overlap.csv")
data

,id,comp,line,start,end,type
0,E001,A,1,2021-01-01,2021-01-05,D
1,E001,A,2,2021-01-03,2021-01-08,D
2,E001,A,3,2020-12-25,2021-01-03,D
3,E001,A,4,2021-01-10,2021-01-12,ND
4,E001,A,5,2021-01-13,2021-01-15,ND
5,E001,A,6,2021-01-01,2021-01-05,D
6,E001,A,7,2021-01-03,2021-01-03,D
7,E001,B,1,2021-01-01,2021-01-05,D
8,E001,B,2,2021-01-03,2021-01-08,D
9,E001,B,3,2020-12-25,2021-01-03,D


# Create **date_handler** function
* This function can compare partial/full dates as per user method.
* if all dates are full dates, it will compare them as usual.
* however if one or more dates in comparison is partial then it will handle those instances as per user method provided.
* Applicable method: (method expected by function)
 * **donothing**: this option on facing the partial dates will do nothing, (means it will not do the compression).
 * **imput**:  this option will use date imputation techniques to handle the intense of partial dates.

* Example of **partial/full** dates
<pre><ul><li> 2020-01-01: full dates</li><li> 2020-01   : partial dates</li><li> 2020      : partial dates</li></ul></pre>





In [ ]:
def date_handler(col_st,col_en,row_st,row_en,method):

  if (len(col_st) in [7,4] or 
      len(col_en) in [7,4] or 
      len(row_st) in [7,4] or 
      len(row_en) in [7,4] ):
     
    if method =="donothing":
          
      return True, "donothing"

    elif method =="imput":
          
      input  = [col_st,col_en,row_st,row_en]
      output = []
      i=0
      for dat in input:
        i+=1
        if len(dat) ==7:
            if  dat.split("-")[1] in ["01","03","05","07","08","10","12"]:
                len7 = "-31"    if i%2 ==0 else "-01"
            elif dat.split("-")[1] == "02":
                len7 = "-28"    if i%2 ==0 else "-01"
            else:
                len7 = "-30"    if i%2 ==0 else "-01"

            out_dat = dat+len7 
        
        elif len(dat) ==4:
            len4 = "-01-31" if i%2 ==0 else "-01-01"
            out_dat = dat+len4 

        elif len(dat) ==10:
            out_dat = dat 

        else:
            out_dat = dat 

        output.append(out_dat)  

    return output[0] <= output[2] <= output[1] or output[0] <= output[3] <= output[1], "imput"

  else:
      return col_st <= row_st <= col_en or col_st <= row_en <= col_en, "full"

# Overlap 
 * This function compares the startdate & enddate of one row to another row to check whether it overlap.
 * Example of overlap:
  * for example if  person (**say E001**) worked on a company(**say  A**) during startdate= **2021-01-01** to enddate= **2021-01-05**  
  * while another record says same person (**say E001**) worked on a company(**say  A**) during startdate= **2021-01-03** to enddate= **2021-01-08**.  

| Person | Company | StartDate  | EndDate  |
|:--:|:--:|:--:|:--:|
| E001 | A | 2021-01-01 | 2021-01-05 |
| E001 | A | 2021-01-03 | 2021-01-08 |

* In the above example you can see dates are overlapping. Person cannot join on 2021-01-03 if he already worked on the same company during 2021-01-01 to 2021-01-05. 

* This type of record is an invalid entry in the database and the objective is to identify this record and clean the database.

In [ ]:
import numpy as np

def Overlap(dataset,subject, drug, logline, startdate, enddate, method):

  dataset[startdate] = dataset[startdate].astype(str).str.slice(0, 10)
  dataset[enddate]   = dataset[enddate].astype(str).str.slice(0, 10)

  for index, row in data.iterrows():
    temp = dataset[(dataset[subject] == row[subject]) & (data[drug] == row[drug])][[logline,startdate,enddate]]
    lst = []

    for idx, col in temp.iterrows():   
      check, typ = date_handler(col[startdate],col[enddate],row[startdate],row[enddate],method) 
      if check== True   and  idx != index:   
        if typ != "donothing":
          lst.append(col[logline]) 

    dataset.loc[index, 'overlap with line'] = ",".join(map(str,lst))
    dataset['is_PartialDate'] = np.where(dataset[startdate].str.len().isin([7,4]) | dataset[enddate].str.len().isin([7,4]),"Partial","Full" )
  return dataset   

In [ ]:
dataset   = data
subject   = "id"
drug      = "comp"
logline   = "line"
startdate = "start"
enddate   = "end"
method    = "imput"

output_data = Overlap(dataset,subject, drug, logline, startdate, enddate, method)

output_data

,id,comp,line,start,end,type,overlap with line,is_PartialDate
0,E001,A,1,2021-01-01,2021-01-05,D,"2,3,6",Full
1,E001,A,2,2021-01-03,2021-01-08,D,"1,3,6,7",Full
2,E001,A,3,2020-12-25,2021-01-03,D,"1,2,6,7",Full
3,E001,A,4,2021-01-10,2021-01-12,ND,,Full
4,E001,A,5,2021-01-13,2021-01-15,ND,,Full
5,E001,A,6,2021-01-01,2021-01-05,D,"1,2,3",Full
6,E001,A,7,2021-01-03,2021-01-03,D,"1,2,3,6",Full
7,E001,B,1,2021-01-01,2021-01-05,D,"2,3,6",Full
8,E001,B,2,2021-01-03,2021-01-08,D,"1,3,6,7",Full
9,E001,B,3,2020-12-25,2021-01-03,D,"1,2,6,7",Full
